In [13]:
import cv2 as cv
import numpy as np
import os
import csv

video_path = "data/videos/test3_87s.mp4"
output_dir = "output/image/templateMatching_SQDIFF_NORMED"
output_csv = "output/txt/templateMatching.csv"
sampling_frequency = 1

# All the 6 methods for comparison in a list
# methods = ['TM_CCOEFF', 'TM_CCOEFF_NORMED', 'TM_CCORR',
#             'TM_CCORR_NORMED', 'TM_SQDIFF', 'TM_SQDIFF_NORMED']

method = getattr(cv, 'TM_SQDIFF_NORMED')
print(method)


cap = cv.VideoCapture(video_path)

# take first frame of the video
ret,frame = cap.read()
assert ret is not False, "video could not be read, check with os.path.exists()"

# get video information
fps = cap.get(cv.CAP_PROP_FPS)
# w, h = frame.shape[::-1]                   
h, w, channels = frame.shape

# Defining region of search
img_w = int(w/4)
img_h = int(h/6)
img_x = int(w/3)
img_y = int(h/2)

# save first frame
frame = cv.rectangle(frame,(img_x, img_y), (img_x + img_w, img_y + img_h), (0, 0, 255), 2)
img_file = os.path.join(output_dir, f"frame_{0}_{method}.jpg")
cv.imwrite(img_file, frame)

# create target (?????)
crop_image = frame[img_y:img_y+img_h, img_x:img_x+img_w]

# Enumerate through video
values = []
y_diff = []
x_diff = []
count = 0
cap.set(cv.CAP_PROP_POS_FRAMES, int((count * fps) / sampling_frequency))        # increment one frame
while (1):
    ret, frame = cap.read()
    if not ret:
        break  # End of video
    
    # frame = frame.astype(np.float32)            # there is something with python's pass by assignment thing here
    # point = frame[0][0]
    # print("template (0, 0) type: ", type(point[0]))
    # print("frame (0, 0) type: ", type(frame[0][0][0]))

    # Apply template matching
    res = cv.matchTemplate(crop_image,frame,method)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)

    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    bottom_right = (top_left[0] + img_w, top_left[1] + img_h)           # how does python scope work???

    # print("Value: ", res[top_left[1], top_left[0]])  # x and y might be flipped again!!!!!!!!!
    values.append(res[top_left[1], top_left[0]])
    y_diff.append(top_left[1] - img_y)
    x_diff.append(top_left[0] - img_x)

    # Increment frame_index
    count += 1
    cap.set(cv.CAP_PROP_POS_FRAMES, int((count * fps) / sampling_frequency))
    crop_image = frame[img_y:img_y+img_h, img_x:img_x+img_w]

    # Drawing output
    frame = cv.rectangle(frame, top_left, bottom_right, 255, 2)
    frame = cv.rectangle(frame,(img_x, img_y), (img_x + img_w, img_y + img_h), (0, 0, 255), 4)
    img_file = os.path.join(output_dir, f"frame_{count}_{method}.jpg")
    cv.imwrite(img_file, frame)
    img_file = os.path.join(output_dir, f"heatmap_{count}_{method}.jpg")
    cv.imwrite(img_file, res)
    
# End
with open(output_csv, 'a', newline='') as csvfile:
    fieldnames = ['index', 'frame', 'value', 'x-diff', 'y-diff']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    # index = 0
    for index in range(count):
        writer.writerow({'index' : index + 1, 
                         'frame' : int(((index + 1)* fps) / sampling_frequency), 
                         'value' : values[index],
                         'x-diff': x_diff[index],
                         'y-diff': y_diff[index]
                         })
        # index += 1

cap.release()

1
